# TO-DO STEPS PISANE NA PIERWSZYCH ZAJĘCIACH

Data Scraping:


## Data Processing (dla obu zbiorów danych)
1. Ułożenie danych w kolumnach.
2. Miejsca zerowe.
3. Feature Engineering. (kolumna koordynaty geograficzne, 


## Data analysis. (+potencjalnie Streamlit)
1. Prócz metrażu czynniki najbardziej wpływające na cenę.
2. Analiza ceny do piętra
3. Analiza odległości geograficznej od centrum
4. Analiza parking/balon
5. Analiza ceny/trendów w konkretnych dzielniach.


# Jeśli model to.
5. Onehotencoding.
6. Train/Test
7. Model.

Bonus points:
    wizualizacja geograficzna (streamlit - prettymap/mapbox)
    stosunek ilości balkonów w mieszkaniu na dzielnice (balkon rate)
    wpływ i częstotliwość '!' w nazwie oferty i ich zestawienie z ceną
    

# TO-DO 16.04.2024

1. Docelowo proponuję, żeby pierwszy etap (pisany w jednym notatniku jupiter) zamknął się po data preparation i data engeenering, po tym etapie zapiszemy dane do plików do dalszych analiz (wielkrotne pobieranie koordynatów może być bolesne). Aby to osiągnąć trzeba:
  - Uspójnić notatniki (na razie jeszcze nie dane :P), tak, byśmy pracowali na jednym.
  - Dodać na początek chaina uzupełnienie zbiorów o nowe dane (można wyciąć fragmenty z utils) - tam, gdzie będziemy analizować dane statycznie i/lub przestrzennie (pierwotny plan).
  - Jeśli idzie o analize w szeregach czasowych, to trochę inne para kaloszy, można zostawić na później -- chyba, że ktorś będzie chciał się tym zająć.
  - Uważam, że trzeba wybrać, które analizy przeprowadzamy na uspójnionych z obu zbiorów danych, a które osobno na zbiorach - otodom ma więcej cech, nieruchomości online (chyba) bardziej rozróżnia pewne cechy (np. te garaże) - Do decyzji.
  - Z obu zbiorów usunąć "duplikaty" - różne ogłoszenia tego samego mieszkania. Tu trzeba będzie przyjąć jakąś heurystykę.
  - Uzupełnić koordynaty
  - Na ich podstawie dodać kolumnę z odległością od wyróżnionego miejsca (centrum) w metrach.
  - Zbiory zapisać


  - Jeśli idzie o szeregi czasowe, to IMO data i cena w badanych punktach czasowych starczy (cokolwiek innego się tam jeszcze mogło zmieniać). Będzie trzeba wybrać tylko te oferty, które zmieniały się w tej kolumnie.

2. Etap analiz:
- Sprawdzić, co z labów Czecha jest do wykorzystania, bo mam przeczycie, że coś z tego, co u nie go robiliśmy się przyda - PCA chociażby i ten (dual) scatterplot.
- Z drugich labów Czecha ten pearson corelation, to macierz korelacji, którą i tak planowaliśmy.
- Na SISI będziemy trenować model drzew decyzyjnych, więc można go tu dodać też.
- Trzeba przejrzeć, co jeszcze planowaliśmy i co jest w "literaturze" (bo znalzałem coś takiego), co jeszcze można przeprowadzić.


3. Wizualizacja:
Poza oczywstymi wykresikami, tabelami itp, ta heatmapa (oraz heatmapa+, jak ze stramlita) wydaje mi się fajnym bajerem. Można spróbować się podjąć. W oógle wydaje mi się, że w tym aspekcie geograficznym można będzie fajnie nie tylko przeprowadzać analizy, ale też pokazywać rzeczy.

Ważna rzecz to dokumentowanie prac (albo w readme, albo zacząć odrębny dokument) nawet, gdy nic nie wychodzi.


To są oczywiście moje wstępne i ogólne pomysły i jeśli macie inne, to śmiało wędrujcie w te strony, które Wam się wydadzą sensowne i słuszne.

    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [20]:
df_otodom = pd.read_csv('datasets/otodom_dataset_raw.csv')

In [21]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   6820 non-null   object
 1   name/title            6820 non-null   object
 2   address               6820 non-null   object
 3   price                 6820 non-null   object
 4   area                  6820 non-null   object
 5   price-per-area        6526 non-null   object
 6   floor/store           6720 non-null   object
 7   no of rooms           6820 non-null   object
 8   year of construction  6154 non-null   object
 9   parking space         4206 non-null   object
 10  market                6154 non-null   object
 11  form of ownership     4941 non-null   object
 12  condition             5229 non-null   object
 13  rent                  2846 non-null   object
 14  heating               4913 non-null   object
 15  advertiser type       6154 non-null   

In [23]:
df_otodom[df_otodom['price-per-area'].isnull()]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
14,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 65m2 + balkon,"ul. Myśliwska, Płaszów, Podgórze, Kraków, mało...",Zapytaj o cenę,"65,56 m²",NaN,1/5,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
18,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 54m2 + balkon Bezpośrednio,"ul. Arsenał, Górka Narodowa, Prądnik Biały, Kr...",Zapytaj o cenę,"54,2 m²",NaN,5/5,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
40,https://www.otodom.pl/pl/oferta/emaus-garden-a...,Emaus Garden | apartament B2_A10,"ul. Piotra Borowego, Zwierzyniec, Zwierzyniec,...",Zapytaj o cenę,"56,43 m²",NaN,2/3,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
47,https://www.otodom.pl/pl/oferta/4-pokojowe-mie...,4-pokojowe mieszkanie 87m2 + balkon Bez Prowizji,"ul. Grzegórzecka 77, Grzegórzki, Grzegórzki, K...",Zapytaj o cenę,"87,95 m²",NaN,2/7,4,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
60,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 68m2 + loggia Bez Prowizji,"ul. Misjonarska, Łobzów, Krowodrza, Kraków, ma...",Zapytaj o cenę,"68,59 m²",NaN,3/7,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2-pokojowe mieszkanie 74m2 + loggia Bez Prowizji,"ul. Składowa, Krowodrza, Krowodrza, Kraków, ma...",Zapytaj o cenę,"74,31 m²",NaN,4/5,2,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6718,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 55m2 + 2 balkony,"ul. Bartosza Głowackiego, Bronowice, Bronowice...",Zapytaj o cenę,"55,08 m²",NaN,parter/4,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
6735,https://www.otodom.pl/pl/oferta/4-pokojowe-mie...,4-pokojowe mieszkanie 73m2 + balkon Bezpośrednio,"Federowicza, Skotniki, Dębniki, Kraków, małopo...",Zapytaj o cenę,"73,11 m²",NaN,1/3,4,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
6742,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2-pokojowe mieszkanie 66m2 + 2 loggie,"ul. Misjonarska, Łobzów, Krowodrza, Kraków, ma...",Zapytaj o cenę,"66,15 m²",NaN,6/7,2,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Braki w danych dotyczących ceny

### Wiersze bez podanej ceny

In [24]:
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_13664\765562517.py:1: SyntaxWarning: invalid escape sequence '\d'
  no_price = df_otodom[~df_otodom['price'].str.contains('\d')]


294

In [25]:
no_price

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
14,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 65m2 + balkon,"ul. Myśliwska, Płaszów, Podgórze, Kraków, mało...",Zapytaj o cenę,"65,56 m²",NaN,1/5,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
18,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 54m2 + balkon Bezpośrednio,"ul. Arsenał, Górka Narodowa, Prądnik Biały, Kr...",Zapytaj o cenę,"54,2 m²",NaN,5/5,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
40,https://www.otodom.pl/pl/oferta/emaus-garden-a...,Emaus Garden | apartament B2_A10,"ul. Piotra Borowego, Zwierzyniec, Zwierzyniec,...",Zapytaj o cenę,"56,43 m²",NaN,2/3,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
47,https://www.otodom.pl/pl/oferta/4-pokojowe-mie...,4-pokojowe mieszkanie 87m2 + balkon Bez Prowizji,"ul. Grzegórzecka 77, Grzegórzki, Grzegórzki, K...",Zapytaj o cenę,"87,95 m²",NaN,2/7,4,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
60,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 68m2 + loggia Bez Prowizji,"ul. Misjonarska, Łobzów, Krowodrza, Kraków, ma...",Zapytaj o cenę,"68,59 m²",NaN,3/7,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2-pokojowe mieszkanie 74m2 + loggia Bez Prowizji,"ul. Składowa, Krowodrza, Krowodrza, Kraków, ma...",Zapytaj o cenę,"74,31 m²",NaN,4/5,2,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6718,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...,3-pokojowe mieszkanie 55m2 + 2 balkony,"ul. Bartosza Głowackiego, Bronowice, Bronowice...",Zapytaj o cenę,"55,08 m²",NaN,parter/4,3,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
6735,https://www.otodom.pl/pl/oferta/4-pokojowe-mie...,4-pokojowe mieszkanie 73m2 + balkon Bezpośrednio,"Federowicza, Skotniki, Dębniki, Kraków, małopo...",Zapytaj o cenę,"73,11 m²",NaN,1/3,4,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,balkon,NaN,NaN
6742,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,2-pokojowe mieszkanie 66m2 + 2 loggie,"ul. Misjonarska, Łobzów, Krowodrza, Kraków, ma...",Zapytaj o cenę,"66,15 m²",NaN,6/7,2,NaN,garaż/miejsce parkingowe,NaN,pełna własność,do wykończenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Usunięcie wierszy z brakującą ceną (no_price)
df_otodom.drop(no_price.index, inplace=True)

In [27]:
df_otodom['price'].count()

6526

In [28]:
# Sprawdzenie, czy wiersze zostały poprawnie usunięte
no_price = df_otodom[~df_otodom['price'].str.contains('\d')]
no_price['price'].count()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_13664\4157452214.py:2: SyntaxWarning: invalid escape sequence '\d'
  no_price = df_otodom[~df_otodom['price'].str.contains('\d')]


0

### Ceny podane w innej walucie

In [29]:
#7 wierszy ma cenę w EUR
different_currency = df_otodom[~df_otodom['price'].str.contains('zł', na=False)]
different_currency['price'].count()

7

In [30]:
df_otodom.loc[different_currency.index]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
1265,https://www.otodom.pl/pl/oferta/penthouse-232m...,Penthouse 232m w zabytkowej kamienicy Stare Mi...,"Nowy Świat, Stare Miasto, Kraków, małopolskie",1 149 999 EUR,"232,1 m²",21 317 zł/m²,4/4,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,gazowe,biuro nieruchomości,nie,NaN,kamienica,cegła
2056,https://www.otodom.pl/pl/oferta/penthouse-w-po...,Penthouse w pobliżu Rynku Głównego,"marszałka JózefaPiłsudskiego, Nowy Świat, Star...",1 150 000 EUR,232 m²,21 326 zł/m²,4/5,9,1906,NaN,wtórny,pełna własność,do zamieszkania,1 700 zł,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
3144,https://www.otodom.pl/pl/oferta/5-pokoi-taras-...,5 pokoi | Taras 77m2 | Komórka | Centrum | Mar...,"ul. marsz. Józefa Piłsudskiego, Nowy Świat, St...",1 149 999 EUR,"232,1 m²",21 317 zł/m²,4/4,5,1908,garaż/miejsce parkingowe,wtórny,pełna własność,do zamieszkania,850 zł,gazowe,biuro nieruchomości,tak,"taras, balkon",kamienica,cegła
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",585 280 EUR,"91,45 m²",27 535 zł/m²,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4722,https://www.otodom.pl/pl/oferta/penthouse-tara...,Penthouse tarasem na dachu w kamienicy z windą,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"228,7 m²",21 634 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,NaN,NaN,biuro nieruchomości,tak,"balkon, taras",kamienica,brak informacji
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",1 150 000 EUR,228 m²,21 700 zł/m²,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,220 EUR,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,cegła
4995,https://www.otodom.pl/pl/oferta/penthouse-star...,Penthouse Stare Miasto II Poziomy. Taras 70m2,"Marszałka Józefa Piłsudskiego, Nowy Świat, Sta...",1 150 000 EUR,"232,1 m²",21 317 zł/m²,4/5,5,1906,NaN,wtórny,NaN,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji


In [31]:
# zamiana cen na wartości liczbowe
df_otodom['price'] = df_otodom['price'].str.replace(' ', '')
df_otodom['price'] = df_otodom['price'].str.extract('(\d+)').astype(float)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_13664\4193540164.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['price'] = df_otodom['price'].str.extract('(\d+)').astype(float)


In [32]:
df_otodom.loc[different_currency.index, 'price']

1265    1149999.0
2056    1150000.0
3144    1149999.0
3977     585280.0
4722    1150000.0
4908    1150000.0
4995    1150000.0
Name: price, dtype: float64

In [33]:
# kurs na 5.04.2024
exchange_rate = 4.28

df_otodom.loc[different_currency.index, 'price'] *= exchange_rate

In [34]:
df_otodom.loc[different_currency.index, 'price']

1265    4921995.72
2056    4922000.00
3144    4921995.72
3977    2504998.40
4722    4922000.00
4908    4922000.00
4995    4922000.00
Name: price, dtype: float64

In [35]:
df_otodom['price'].count()

6526

### Zamiana wartości w kolumnach na typ liczbowy

In [36]:
df_otodom['area'] = df_otodom['area'].str.replace(' m²', '')
df_otodom['area'] = df_otodom['area'].str.replace(',', '.').astype(float)

In [37]:
df_otodom['area']

0       55.98
1       87.00
2       47.10
3       66.69
4       65.81
        ...  
6815    66.86
6816    67.56
6817    47.35
6818    36.09
6819    73.74
Name: area, Length: 6526, dtype: float64

In [38]:
df_otodom[df_otodom['price-per-area'].str.contains(r'\d', na=False) & ~df_otodom['price-per-area'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material


In [39]:
df_otodom[df_otodom['price-per-area'].isnull()]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material


In [40]:
df_otodom['price-per-area'] = df_otodom['price-per-area'].str.replace(' zł/m²', '').str.replace(' ', '')
df_otodom['price-per-area'] = df_otodom['price-per-area'].astype(float)

In [41]:
#trzy wiersze w innej walucie
df_otodom[df_otodom['rent'].str.contains(r'\d', na=False) & ~df_otodom['rent'].str.contains('zł', na=False)]

,url,name/title,address,price,area,price-per-area,floor/store,no of rooms,year of construction,parking space,market,form of ownership,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material
3977,https://www.otodom.pl/pl/oferta/chorwacja-spli...,Chorwacja Split luksusowy apartament,"Krowodrza, Krowodrza, Kraków, małopolskie",2504998.4,91.45,27535.0,3/4,4,2025,garaż/miejsce parkingowe,pierwotny,pełna własność,do zamieszkania,100 EUR,elektryczne,prywatny,tak,"balkon, taras",apartamentowiec,silikat
4908,https://www.otodom.pl/pl/oferta/penthouse-w-od...,Penthouse w odrestaurowanej kamienicy - taras ...,"ul. marsz. Józefa Piłsudskiego, Piasek, Stare ...",4922000.0,228.00,21700.0,4/5,5,1906,NaN,wtórny,pełna własność,do zamieszkania,220 EUR,gazowe,biuro nieruchomości,tak,"balkon, taras",kamienica,cegła
6335,https://www.otodom.pl/pl/oferta/2m-wysoki-stan...,2M! Wysoki Standard| Gotowe| Balkon | Widok!,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",620000.0,35.55,17440.0,7/11,2,2019,NaN,wtórny,pełna własność,do zamieszkania,420 GBP,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji


In [42]:
eur_rent_price = df_otodom[df_otodom['rent'].str.contains('EUR', na=False)]
gbp_rent_price = df_otodom[df_otodom['rent'].str.contains('GBP', na=False)]

In [43]:
# zamiana wartości rent na typ liczbowy
df_otodom['rent'] = df_otodom['rent'].str.extract('(\d+)').astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_13664\1831358940.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['rent'] = df_otodom['rent'].str.extract('(\d+)').astype(float)


In [44]:
euro_exchange_rate = 4.28
gbp_exchange_rate = 4.99

df_otodom.loc[eur_rent_price.index, 'rent'] *= euro_exchange_rate
df_otodom.loc[gbp_rent_price.index, 'rent'] *= gbp_exchange_rate

In [45]:
df_otodom.loc[eur_rent_price.index, 'rent']

3977    428.0
4908    941.6
Name: rent, dtype: float64

In [46]:
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.replace(' ', '')
df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\katar\AppData\Local\Temp\ipykernel_13664\844144124.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_otodom['no of rooms'] = df_otodom['no of rooms'].str.extract('(\d+)').astype(int)


In [47]:
# Rozdzielenie kolumny 'floor/store' na dwie osobne kolumny
df_otodom[['floor', 'store']] = df_otodom['floor/store'].str.split('/', expand=True)

# Usunięcie oryginalnej kolumny 'floor/store'
df_otodom.drop(columns=['floor/store'], inplace=True)


In [48]:
df_otodom[df_otodom['store'].isnull()]

,url,name/title,address,price,area,price-per-area,no of rooms,year of construction,parking space,market,...,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,floor,store
42,https://www.otodom.pl/pl/oferta/ok-lasu-borkow...,"Ok. Lasu Borkow. piękne, duże mieszkanie, 2010r.","ul. Gwieździsta, Ruczaj, Dębniki, Kraków, mało...",950000.0,70.00,13571.0,3,2010,NaN,wtórny,...,do zamieszkania,450.0,gazowe,biuro nieruchomości,nie,NaN,blok,brak informacji,NaN,NaN
49,https://www.otodom.pl/pl/oferta/pieciopokojowe...,Pięciopokojowe 116m2 bez 2% Pcc,"ul. Miarowa, Swoszowice, Swoszowice, Kraków, m...",1300000.0,116.00,11207.0,5,2020,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,taras,blok,brak informacji,NaN,NaN
91,https://www.otodom.pl/pl/oferta/dom-z-ogrodkie...,Dom z ogródkiem | Dwa m. postojowe | gotowe,"Prądnik Biały, Prądnik Biały, Kraków, małopolskie",759000.0,76.70,9896.0,4,2023,garaż/miejsce parkingowe,pierwotny,...,NaN,NaN,gazowe,biuro nieruchomości,nie,NaN,brak informacji,pustak,NaN,NaN
96,https://www.otodom.pl/pl/oferta/okazja-ladne-m...,Okazja ! Ładne Mieszkanie Bezczynszowe,"Bronowice Wielkie, Prądnik Biały, Kraków, mało...",490000.0,34.00,14412.0,2,brak informacji,garaż/miejsce parkingowe,wtórny,...,do wykończenia,NaN,elektryczne,prywatny,nie,ogródek,szeregowiec,cegła,parter,None
126,https://www.otodom.pl/pl/oferta/jasne-mieszkan...,Jasne mieszkanie 44 m2 przy ul. Wrocławskiej 53,"ul. Wrocławska, Krowodrza, Krowodrza, Kraków, ...",875000.0,44.00,19886.0,2,brak informacji,garaż/miejsce parkingowe,wtórny,...,do zamieszkania,700.0,miejskie,biuro nieruchomości,tak,balkon,brak informacji,brak informacji,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6632,https://www.otodom.pl/pl/oferta/nowa-huta-3-po...,"Nowa Huta, 3 pokoje, po remoncie 57m","Nowa Huta, Nowa Huta, Kraków, małopolskie",799000.0,57.00,14018.0,3,1952,NaN,wtórny,...,do zamieszkania,600.0,miejskie,biuro nieruchomości,nie,NaN,blok,cegła,3,None
6694,https://www.otodom.pl/pl/oferta/przestronne-mi...,Przestronne mieszkanie,"ul. Stańczyka, Bronowice, Bronowice, Kraków, m...",680000.0,36.56,18600.0,2,2020,NaN,wtórny,...,do zamieszkania,490.0,NaN,prywatny,tak,balkon,blok,brak informacji,2,None
6706,https://www.otodom.pl/pl/oferta/nowy-apartamen...,NOWy apartament z dużym ogrodem Olszanica,"ul. Olszanicka, Chełm, Zwierzyniec, Kraków, ma...",1195000.0,111.00,10766.0,5,2024,NaN,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,"balkon, ogródek",apartamentowiec,pustak,NaN,NaN
6715,https://www.otodom.pl/pl/oferta/krakow-ksiecia...,Kraków Księcia Józefa mieszkanie,"ul. Astronautów, Bielany, Zwierzyniec, Kraków,...",485000.0,39.00,12436.0,1,brak informacji,NaN,wtórny,...,do zamieszkania,270.0,gazowe,prywatny,nie,NaN,blok,cegła,2,None


In [49]:
df_otodom[df_otodom['floor'].isnull()]

,url,name/title,address,price,area,price-per-area,no of rooms,year of construction,parking space,market,...,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,floor,store
42,https://www.otodom.pl/pl/oferta/ok-lasu-borkow...,"Ok. Lasu Borkow. piękne, duże mieszkanie, 2010r.","ul. Gwieździsta, Ruczaj, Dębniki, Kraków, mało...",950000.0,70.00,13571.0,3,2010,NaN,wtórny,...,do zamieszkania,450.0,gazowe,biuro nieruchomości,nie,NaN,blok,brak informacji,NaN,NaN
49,https://www.otodom.pl/pl/oferta/pieciopokojowe...,Pięciopokojowe 116m2 bez 2% Pcc,"ul. Miarowa, Swoszowice, Swoszowice, Kraków, m...",1300000.0,116.00,11207.0,5,2020,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,taras,blok,brak informacji,NaN,NaN
91,https://www.otodom.pl/pl/oferta/dom-z-ogrodkie...,Dom z ogródkiem | Dwa m. postojowe | gotowe,"Prądnik Biały, Prądnik Biały, Kraków, małopolskie",759000.0,76.70,9896.0,4,2023,garaż/miejsce parkingowe,pierwotny,...,NaN,NaN,gazowe,biuro nieruchomości,nie,NaN,brak informacji,pustak,NaN,NaN
144,https://www.otodom.pl/pl/oferta/piekne-mieszka...,"Piękne mieszkanie dla rodziny z ogródkiem, 202...","Kobierzyn, Dębniki, Kraków, małopolskie",820000.0,64.00,12813.0,3,2020,NaN,wtórny,...,do zamieszkania,700.0,miejskie,biuro nieruchomości,nie,ogródek,blok,brak informacji,NaN,NaN
221,https://www.otodom.pl/pl/oferta/mieszkanie-z-o...,"Mieszkanie z ogródkiem w Swoszowicach - 78,45 m2","Swoszowice, Swoszowice, Kraków, małopolskie",749000.0,78.45,9547.0,4,2023,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,NaN,brak informacji,silikat,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,https://www.otodom.pl/pl/oferta/51m2-nowa-huta...,"51m2, Nowa Huta, Ogródek, Po generalnych remoncie","os. Krakowiaków, Nowa Huta, Nowa Huta, Kraków,...",790000.0,50.58,15619.0,2,1955,NaN,wtórny,...,do zamieszkania,750.0,miejskie,biuro nieruchomości,nie,NaN,blok,brak informacji,NaN,NaN
6418,https://www.otodom.pl/pl/oferta/mieszkanie-2-p...,"Mieszkanie 2-pokojowe | 35,8 m2 | Azory","Azory, Prądnik Biały, Kraków, małopolskie",536000.0,35.80,14972.0,2,1977,NaN,wtórny,...,NaN,457.0,miejskie,biuro nieruchomości,nie,NaN,blok,wielka płyta,NaN,NaN
6439,https://www.otodom.pl/pl/oferta/3-pok-kuchnia-...,"3 pok.+ kuchnia, udziały w nieruchomości, Pilotów","ul. Widna, Rakowice, Prądnik Czerwony, Kraków,...",365000.0,65.00,5615.0,3,1960,NaN,wtórny,...,do zamieszkania,30.0,gazowe,biuro nieruchomości,nie,ogródek,brak informacji,brak informacji,NaN,NaN
6706,https://www.otodom.pl/pl/oferta/nowy-apartamen...,NOWy apartament z dużym ogrodem Olszanica,"ul. Olszanicka, Chełm, Zwierzyniec, Kraków, ma...",1195000.0,111.00,10766.0,5,2024,NaN,pierwotny,...,do wykończenia,NaN,gazowe,biuro nieruchomości,nie,"balkon, ogródek",apartamentowiec,pustak,NaN,NaN


In [50]:
df_otodom = df_otodom.dropna(subset=['floor', 'store'])

In [51]:
df_otodom.head()

,url,name/title,address,price,area,price-per-area,no of rooms,year of construction,parking space,market,...,condition,rent,heating,advertiser type,elevator,outdoor area,building type,building material,floor,store
0,https://www.otodom.pl/pl/oferta/gotowe-2-pokoj...,Gotowe| 2 pokoje| blisko centrum| Bonarka,"Wola Duchacka, Podgórze Duchackie, Kraków, mał...",719000.0,55.98,12844.0,3,2022,NaN,pierwotny,...,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,ogródek,apartamentowiec,brak informacji,parter,8
1,https://www.otodom.pl/pl/oferta/4-pok-mieszkan...,4-pok.mieszkanie z Sauną - Wysoki Standard ! 2...,"Biskupa Albina Małysiaka, Kobierzyn, Dębniki, ...",1350000.0,87.00,15517.0,4,2021,garaż/miejsce parkingowe,wtórny,...,NaN,900.0,miejskie,biuro nieruchomości,tak,balkon,brak informacji,cegła,2,4
2,https://www.otodom.pl/pl/oferta/3-pokoje-w-rza...,3 pokoje w rządowym programie kredyt 2%,"Bieżanów, Bieżanów-Prokocim, Kraków, małopolskie",599000.0,47.10,12718.0,3,2023,garaż/miejsce parkingowe,pierwotny,...,do wykończenia,NaN,miejskie,biuro nieruchomości,tak,balkon,blok,brak informacji,1,4
3,https://www.otodom.pl/pl/oferta/mieszkanie-ide...,"Mieszkanie idealne na start, 3 - pokoje!","ul. Erazma Jerzmanowskiego, Prokocim, Bieżanów...",833500.0,66.69,12498.0,3,2024,garaż/miejsce parkingowe,pierwotny,...,NaN,NaN,miejskie,biuro nieruchomości,tak,balkon,apartamentowiec,brak informacji,3,8
4,https://www.otodom.pl/pl/oferta/ul-lasowka-3-p...,"ul. Lasówka, 3 pokoje, 65m2 + taras 20m2!","ul. Lasówka, Płaszów, Podgórze, Kraków, małopo...",949000.0,65.81,14420.0,3,2023,NaN,wtórny,...,NaN,NaN,NaN,biuro nieruchomości,nie,NaN,brak informacji,brak informacji,7,7


In [52]:
# Zamiana wartości "parter" na 0 w kolumnie "floor"
df_otodom['floor'] = df_otodom['floor'].replace('parter', 0)

In [53]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor"
df_otodom['floor'] = df_otodom['floor'].replace('suterena', -1)

In [54]:
# Konwersja kolumny "floor" na typ liczbowy, zamieniając nieliczbowe wartości na NaN
df_otodom['floor'] = pd.to_numeric(df_otodom['floor'], errors='coerce')

# Usunięcie wierszy zawierających wartości NaN w kolumnie "floor"
df_otodom = df_otodom[~df_otodom['floor'].isna()]

In [55]:
df_otodom['floor'] = df_otodom['floor'].astype(int)

In [56]:
df_otodom['store'] = df_otodom['store'].astype(int)

In [57]:
df_otodom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6112 entries, 0 to 6819
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   6112 non-null   object 
 1   name/title            6112 non-null   object 
 2   address               6112 non-null   object 
 3   price                 6112 non-null   float64
 4   area                  6112 non-null   float64
 5   price-per-area        6112 non-null   float64
 6   no of rooms           6112 non-null   int32  
 7   year of construction  5704 non-null   object 
 8   parking space         3681 non-null   object 
 9   market                5704 non-null   object 
 10  form of ownership     4372 non-null   object 
 11  condition             4639 non-null   object 
 12  rent                  2697 non-null   float64
 13  heating               4567 non-null   object 
 14  advertiser type       5704 non-null   object 
 15  elevator              5704

In [58]:
df_otodom['price'].mean()

985104.6141753927

In [59]:
df_otodom.describe()

,price,area,price-per-area,no of rooms,rent,floor,store
count,6.112000e+03,6112.000000,6112.000000,6112.000000,2697.000000,6112.000000,6112.000000
mean,9.851046e+05,59.900411,16574.702716,2.698135,480.097293,2.282886,4.675393
std,6.599385e+05,32.705208,4520.633846,1.059980,235.621399,2.008813,2.577294
min,9.500000e+04,10.270000,271.000000,1.000000,0.000000,-1.000000,1.000000
25%,6.650000e+05,41.000000,13749.750000,2.000000,350.000000,1.000000,3.000000
50%,8.344000e+05,53.000000,15984.000000,3.000000,500.000000,2.000000,4.000000
75%,1.080000e+06,68.790000,18500.000000,3.000000,650.000000,3.000000,6.000000
max,1.350000e+07,706.000000,70000.000000,10.000000,2095.800000,10.000000,93.000000


In [63]:
df_otodom.loc[df_otodom['price'].idxmax()]

url                     https://www.otodom.pl/pl/oferta/najpiekniejszy...
name/title                           Najpiękniejszy Apartament w Krakowie
address                 ul. Koletek, Stradom, Stare Miasto, Kraków, ma...
price                                                          13500000.0
area                                                                270.0
price-per-area                                                    50000.0
no of rooms                                                             4
year of construction                                                 2016
parking space                                    garaż/miejsce parkingowe
market                                                             wtórny
form of ownership                                                     NaN
condition                                                 do zamieszkania
rent                                                                  NaN
heating                               

## Zbiór danych nieruchomosci-online

In [340]:
df_nieruchomosci = pd.read_csv('datasets/nieruchomosci-online_dataset_raw.csv')

In [341]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3949 entries, 0 to 3948
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  3949 non-null   object 
 1   name/title                           3949 non-null   object 
 2   address                              3848 non-null   object 
 3   price                                3920 non-null   object 
 4   area                                 3949 non-null   object 
 5   price-per-area                       3920 non-null   object 
 6   floor/store                          3949 non-null   object 
 7   no of floors/stores in the building  3710 non-null   float64
 8   no of rooms                          3949 non-null   object 
 9   year of construction                 3949 non-null   object 
 10  parking space                        3949 non-null   object 
 11  market                        

### Usunięcie wierszy z brakującymi danymi

In [370]:
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['price'].isnull()].index, inplace=True)

In [371]:
df_nieruchomosci[df_nieruchomosci['price'].isnull()]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership


In [372]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3864 entries, 0 to 3948
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  3864 non-null   object 
 1   name/title                           3864 non-null   object 
 2   address                              3802 non-null   object 
 3   price                                3864 non-null   float64
 4   area                                 3864 non-null   float64
 5   price-per-area                       3864 non-null   float64
 6   floor/store                          3864 non-null   object 
 7   no of floors/stores in the building  3703 non-null   float64
 8   no of rooms                          3864 non-null   int32  
 9   year of construction                 3864 non-null   object 
 10  parking space                        3864 non-null   object 
 11  market                             

In [373]:
df_nieruchomosci.head(20)

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
0,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Żelechowskiego","Żelechowskiego, Bronowice, Kraków, małopolskie",899000.0,51.70,17388.0,1,3.0,2,1980,tak,wtórny,własność
1,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Balicka","Balicka, Bronowice, Kraków, małopolskie",1575000.0,125.00,12600.0,3,4.0,6,2004,w garażu podziemnym,wtórny,"własność, księga wieczysta"
2,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Zauchy","Zauchy, Górka Narodowa, Kraków, małopolskie",1250000.0,65.23,19162.0,2,5.0,3,2023,tak,wtórny,NaN
3,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Racławicka","Racławicka, Krowodrza, Kraków, małopolskie",740250.0,49.35,15000.0,6,10.0,2,1950,parking publiczny / na ulicy,wtórny,NaN
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.0,0,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1198680.0,71.35,16800.0,0,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
6,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Przemiarki","Przemiarki, Ruczaj, Kraków, małopolskie",625000.0,40.00,15625.0,2,3.0,2,2000,garaż,wtórny,własność
7,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Felińskiego","Felińskiego, Górka Narodowa, Kraków, małopolskie",899000.0,74.17,12120.0,2,5.0,3,2003,parking publiczny / na ulicy,wtórny,NaN
8,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Lublańska","Lublańska 13, Prądnik Czerwony, Kraków, małopo...",1050000.0,60.00,17500.0,3,4.0,3,2023,w garażu podziemnym,wtórny,"własność, księga wieczysta"
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599000.0,40.00,14975.0,0,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"


In [374]:
# sprawdzenie, czy są jakieś wartości w kolumnie price-per-area, które nie zawierają danych liczbowych
df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains(r'\d')]

AttributeError: Can only use .str accessor with string values!

In [ ]:
# sprawdzenie, czy są jakieś wartości w kolumnie price, które nie zawierają danych liczbowych
df_nieruchomosci[~df_nieruchomosci['price'].str.contains(r'\d')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership


### Zamiana cen podanych w innych walutach

In [ ]:
# wartości w kolumnie 'price' podane w innej walucie
different_currency_price = df_nieruchomosci[~df_nieruchomosci['price'].str.contains('zł')]
different_currency_price

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
153,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1906,parking strzeżony w pobliżu,wtórny,NaN
1120,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1908,garaż,wtórny,NaN
1151,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,"232,10 m²","4 954,76 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,NaN
2096,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,228 m²,"5 043,86 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,własność
2241,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Nowy Świat, Kraków, małopolskie",1 150 000 €,"228,70 m²","5 028,42 €/m²",4,5.0,5,1906,-,wtórny,NaN
2562,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Kraków, małopolskie",138 000 €,65 m²,"2 123,08 €/m²",21,45.0,2,2023,-,wtórny,NaN
2927,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, Plac Juliusza Kossaka","Plac Juliusza Kossaka, Stare Miasto, Kraków, m...",700 000 €,152 m²,"4 605,26 €/m²",1,4.0,4,1930,-,wtórny,NaN
2991,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"marszałka JózefaPiłsudskiego, Wawel, Kraków, m...",1 150 000 €,232 m²,"4 956,90 €/m²",4,5.0,9,1906,-,wtórny,NaN
3364,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie Stare Miasto, ul. Jana","Jana, Stare Miasto, Kraków, małopolskie",1 000 000 €,"221,19 m²",4 521 €/m²,4,4.0,4,1850,-,wtórny,NaN


In [ ]:
# price-per-area w innej walucie
df_nieruchomosci[~df_nieruchomosci['price-per-area'].str.contains('zł')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
153,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1906,parking strzeżony w pobliżu,wtórny,NaN
1120,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 149 999 €,"232,10 m²","4 954,76 €/m²",4,4.0,5,1908,garaż,wtórny,NaN
1151,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,"232,10 m²","4 954,76 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,NaN
2096,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Stare Miasto, Kraków, małopolskie",1 150 000 €,228 m²,"5 043,86 €/m²",4,5.0,5,1906,parking publiczny / na ulicy,wtórny,własność
2241,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Piłsudskiego","Piłsudskiego, Nowy Świat, Kraków, małopolskie",1 150 000 €,"228,70 m²","5 028,42 €/m²",4,5.0,5,1906,-,wtórny,NaN
2562,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Kraków, małopolskie",138 000 €,65 m²,"2 123,08 €/m²",21,45.0,2,2023,-,wtórny,NaN
2927,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, Plac Juliusza Kossaka","Plac Juliusza Kossaka, Stare Miasto, Kraków, m...",700 000 €,152 m²,"4 605,26 €/m²",1,4.0,4,1930,-,wtórny,NaN
2991,https://krakow.nieruchomosci-online.pl/mieszka...,Mieszkanie Kraków,"marszałka JózefaPiłsudskiego, Wawel, Kraków, m...",1 150 000 €,232 m²,"4 956,90 €/m²",4,5.0,9,1906,-,wtórny,NaN
3364,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie Stare Miasto, ul. Jana","Jana, Stare Miasto, Kraków, małopolskie",1 000 000 €,"221,19 m²",4 521 €/m²,4,4.0,4,1850,-,wtórny,NaN


In [ ]:
# zamiana cen na wartości liczbowe
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.replace(' ', '')
df_nieruchomosci['price'] = df_nieruchomosci['price'].str.extract(r'(\d+)').astype(float)

df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.replace(' ', '')
df_nieruchomosci['price-per-area'] = df_nieruchomosci['price-per-area'].str.extract(r'(\d+)').astype(float)

In [ ]:
df_nieruchomosci.loc[different_currency.index, ['price', 'price-per-area']]

,price,price-per-area
153,1149999.0,4954.0
1120,1149999.0,4954.0
1151,1150000.0,4954.0
2096,1150000.0,5043.0
2241,1150000.0,5028.0
2562,138000.0,2123.0
2927,700000.0,4605.0
2991,1150000.0,4956.0
3364,1000000.0,4521.0


In [ ]:
# kurs na 5.04.2024
euro_exchange_rate = 4.28

df_nieruchomosci.loc[different_currency.index, 'price'] *= euro_exchange_rate
df_nieruchomosci.loc[different_currency.index, 'price-per-area'] *= euro_exchange_rate

In [ ]:
df_nieruchomosci.loc[different_currency.index, ['price', 'price-per-area']]

,price,price-per-area
153,4921995.72,21203.12
1120,4921995.72,21203.12
1151,4922000.00,21203.12
2096,4922000.00,21584.04
2241,4922000.00,21519.84
2562,590640.00,9086.44
2927,2996000.00,19709.40
2991,4922000.00,21211.68
3364,4280000.00,19349.88


### Przesunięcie danych w kolumnach

In [ ]:
# usunięcie wierszy z brakującymi danymi (wartośc "-")
df_nieruchomosci.drop(df_nieruchomosci[df_nieruchomosci['no of rooms'].str.contains('-')].index, inplace=True)

In [ ]:
# niektóre mieszkania zamiast liczby pokoi mają wpisany poziom mieszkania
shifted_data = df_nieruchomosci[~df_nieruchomosci['no of rooms'].str.contains(r'\d')]
shifted_data.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
37,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1373420.0,"79,85 m²",17200.0,II kwartał 2024,NaN,parter,4,-,II kwartał 2024,pierwotny (oferta dewelopera)
71,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Bronowice Wielkie, ul. Piaskow...",NaN,903189.0,"73,43 m²",12300.0,oddana do użytku,NaN,parter,3,naziemne,"IV kwartał 2022, oddana do użytku",pierwotny (oferta dewelopera)
119,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wola Justowska, ul. Królowej J...",NaN,1221192.0,"72,69 m²",16800.0,zrealizowana,NaN,parter,3,garaż,"październik 2023, zrealizowana",pierwotny (oferta dewelopera)
178,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Tonie, ul. Władysława Łokietka",NaN,1638000.0,"136,60 m²",11991.0,październik 2025,NaN,parter,4,garaż,październik 2025,pierwotny (oferta dewelopera)
212,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Prądnik Biały, Piaszczysta/Pęk...",NaN,917868.0,"62,44 m²",14700.0,IV kwartał 2024,NaN,parter,3,garaż,IV kwartał 2024,pierwotny (oferta dewelopera)


In [ ]:
no_of_rooms = df_nieruchomosci.loc[shifted_data.index, 'year of construction']
floor = df_nieruchomosci.loc[shifted_data.index, 'no of rooms']
year = df_nieruchomosci.loc[shifted_data.index, 'market']
market = df_nieruchomosci.loc[shifted_data.index, 'form of ownership']

In [ ]:
df_nieruchomosci.loc[shifted_data.index, 'floor/store'] = floor
df_nieruchomosci.loc[shifted_data.index, 'no of rooms'] = no_of_rooms
df_nieruchomosci.loc[shifted_data.index, 'year of construction'] = year
df_nieruchomosci.loc[shifted_data.index, 'market'] = market

In [ ]:
df_nieruchomosci.loc[shifted_data.index, 'form of ownership'] = np.nan

In [ ]:
df_nieruchomosci.loc[shifted_data.index].head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
37,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1373420.0,"79,85 m²",17200.0,parter,NaN,4,II kwartał 2024,-,pierwotny (oferta dewelopera),NaN
71,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Bronowice Wielkie, ul. Piaskow...",NaN,903189.0,"73,43 m²",12300.0,parter,NaN,3,"IV kwartał 2022, oddana do użytku",naziemne,pierwotny (oferta dewelopera),NaN
119,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wola Justowska, ul. Królowej J...",NaN,1221192.0,"72,69 m²",16800.0,parter,NaN,3,"październik 2023, zrealizowana",garaż,pierwotny (oferta dewelopera),NaN
178,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Tonie, ul. Władysława Łokietka",NaN,1638000.0,"136,60 m²",11991.0,parter,NaN,4,październik 2025,garaż,pierwotny (oferta dewelopera),NaN
212,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Prądnik Biały, Piaszczysta/Pęk...",NaN,917868.0,"62,44 m²",14700.0,parter,NaN,3,IV kwartał 2024,garaż,pierwotny (oferta dewelopera),NaN


### Zamiana wartości w kolumnach na typ liczbowy

In [ ]:
df_nieruchomosci['no of rooms'] = df_nieruchomosci['no of rooms'].astype(int)

In [ ]:
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(' m²', '')
df_nieruchomosci['area'] = df_nieruchomosci['area'].str.replace(',', '.').astype(float)

In [ ]:
df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains('parter')]

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
4,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Pustynna","Pustynna, Kliny, Kraków, małopolskie",1200000.0,71.06,16887.0,parter,1.0,3,2013,garaż w bryle budynku,wtórny,"własność, księga wieczysta"
5,https://krakow.nieruchomosci-online.pl/mieszka...,Apartament Kraków,"Wola Justowska, Kraków, małopolskie",1198680.0,71.35,16800.0,parter,2.0,4,2023,garaż w bryle budynku,pierwotny (zobacz inne nowe mieszkania w Krako...,własność
9,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Szablowskiego","Szablowskiego, Bronowice, Kraków, małopolskie",599000.0,40.00,14975.0,parter,3.0,2,2001,tak,wtórny,"własność, księga wieczysta"
12,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Ułanów","Ułanów, Prądnik Czerwony, Kraków, małopolskie",790000.0,61.40,12866.0,parter,10.0,3,1962,-,wtórny,"spółdzielcze własnościowe, księga wieczysta"
18,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Obozowa","Obozowa, Ruczaj, Kraków, małopolskie",695000.0,51.94,13380.0,parter,7.0,2,2012,przynależne na ulicy,wtórny,"własność, księga wieczysta"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Mogilska","Mogilska, Grzegórzki, Kraków, małopolskie",703380.0,37.02,19000.0,parter,4.0,2,2024,parking publiczny / na ulicy,wtórny,NaN
3927,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Na Błonie","Na Błonie, Bronowice Małe, Kraków, małopolskie",615000.0,47.00,13085.0,parter,10.0,3,1970,tak,wtórny,własność
3930,https://krakow.nieruchomosci-online.pl/mieszka...,"Apartament, ul. Madalińskiego","Dębniki, Kraków, małopolskie",894375.0,56.25,15900.0,parter,3.0,3,2020,parking publiczny / na ulicy,wtórny,NaN
3932,https://krakow.nieruchomosci-online.pl/mieszka...,"Mieszkanie, ul. Starego Dębu","Starego Dębu, Bronowice Wielkie, Kraków, małop...",799000.0,68.00,11750.0,parter,4.0,2,2019,-,wtórny,NaN


In [ ]:
# Zamiana wartości "parter" na 0 w kolumnie "floor/store"
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('parter', '0')

In [ ]:
# Zamiana wartości "suterena" na -1 w kolumnie "floor/store"
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].replace('suterena', '-1')

In [ ]:
# usunięcie wierszy, które w kolumnie 'floor/store' zawierają dane nieliczbowe
df_nieruchomosci.drop(df_nieruchomosci[~df_nieruchomosci['floor/store'].str.contains(r'\d')].index, inplace=True)

In [379]:
# przesunięte dane
shifted_data = df_nieruchomosci[df_nieruchomosci['floor/store'].str.contains(r'[a-zA-Z]')]
shifted_data.head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
35,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1501610.0,80.30,18700.0,II kwartał 2024,NaN,4,4,-,II kwartał 2024,pierwotny (oferta dewelopera)
39,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1883130.0,91.86,20500.0,II kwartał 2024,NaN,3,4,-,II kwartał 2024,pierwotny (oferta dewelopera)
41,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,912966.0,77.37,11800.0,III kwartał 2024,NaN,2,3,garaż,III kwartał 2024,pierwotny (oferta dewelopera)
42,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,1599000.0,121.37,13174.0,III kwartał 2024,NaN,2,5,garaż,III kwartał 2024,pierwotny (oferta dewelopera)
81,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Prokocim, ul. Republiki Korcza...",NaN,630878.0,41.78,15100.0,II kwartał 2025,NaN,7,2,garaż,II kwartał 2025,pierwotny (oferta dewelopera)


In [380]:
no_of_rooms = df_nieruchomosci.loc[shifted_data.index, 'year of construction']
floor = df_nieruchomosci.loc[shifted_data.index, 'no of rooms']
year = df_nieruchomosci.loc[shifted_data.index, 'market']
market = df_nieruchomosci.loc[shifted_data.index, 'form of ownership']

In [381]:
df_nieruchomosci.loc[shifted_data.index, 'floor/store'] = floor
df_nieruchomosci.loc[shifted_data.index, 'no of rooms'] = no_of_rooms
df_nieruchomosci.loc[shifted_data.index, 'year of construction'] = year
df_nieruchomosci.loc[shifted_data.index, 'market'] = market

C:\Users\katar\AppData\Local\Temp\ipykernel_13664\502791243.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['4' '4' '3' '5' '2' '3' '2' '2' '4' '3' '3' '3' '3' '3' '3' '3' '2' '4'
 '2' '2' '3' '4' '3' '4' '4' '2' '2' '2' '2' '3' '2' '2' '3' '2' '4' '3'
 '3' '4' '4' '2' '2' '3']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_nieruchomosci.loc[shifted_data.index, 'no of rooms'] = no_of_rooms


In [382]:
df_nieruchomosci.loc[shifted_data.index, 'form of ownership'] = np.nan

In [383]:
df_nieruchomosci.loc[shifted_data.index].head()

,url,name/title,address,price,area,price-per-area,floor/store,no of floors/stores in the building,no of rooms,year of construction,parking space,market,form of ownership
35,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1501610.0,80.30,18700.0,4,NaN,4,II kwartał 2024,-,pierwotny (oferta dewelopera),NaN
39,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Wieczysta, Aleja Jana Pawła II 52",NaN,1883130.0,91.86,20500.0,3,NaN,4,II kwartał 2024,-,pierwotny (oferta dewelopera),NaN
41,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,912966.0,77.37,11800.0,2,NaN,3,III kwartał 2024,garaż,pierwotny (oferta dewelopera),NaN
42,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Gotyk, ul. Grabczaka 8",NaN,1599000.0,121.37,13174.0,2,NaN,5,III kwartał 2024,garaż,pierwotny (oferta dewelopera),NaN
81,https://krakow.nieruchomosci-online.pl/nowe-mi...,"Nowe mieszkanie Prokocim, ul. Republiki Korcza...",NaN,630878.0,41.78,15100.0,7,NaN,2,II kwartał 2025,garaż,pierwotny (oferta dewelopera),NaN


In [384]:
df_nieruchomosci['floor/store'] = df_nieruchomosci['floor/store'].astype(int)

In [386]:
df_nieruchomosci.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3864 entries, 0 to 3948
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   url                                  3864 non-null   object 
 1   name/title                           3864 non-null   object 
 2   address                              3802 non-null   object 
 3   price                                3864 non-null   float64
 4   area                                 3864 non-null   float64
 5   price-per-area                       3864 non-null   float64
 6   floor/store                          3864 non-null   int32  
 7   no of floors/stores in the building  3703 non-null   float64
 8   no of rooms                          3864 non-null   object 
 9   year of construction                 3864 non-null   object 
 10  parking space                        3864 non-null   object 
 11  market                             

In [385]:
df_nieruchomosci.describe()

,price,area,price-per-area,floor/store,no of floors/stores in the building
count,3.864000e+03,3864.000000,3864.000000,3864.000000,3703.000000
mean,1.027597e+06,61.522873,16860.762329,2.374224,4.851472
std,7.296519e+05,33.563703,4952.155442,2.227826,2.532037
min,1.490000e+05,12.300000,3000.000000,-1.000000,1.000000
25%,6.801305e+05,41.760000,13821.000000,1.000000,3.000000
50%,8.402940e+05,54.580000,16117.500000,2.000000,4.000000
75%,1.115000e+06,69.630000,18970.000000,4.000000,6.000000
max,1.600000e+07,550.000000,71018.000000,21.000000,45.000000


In [ ]:
df_nieruchomosci.isnull().sum()

url                                       0
name/title                                0
address                                  74
price                                     0
area                                      0
price-per-area                            0
floor/store                               0
no of floors/stores in the building     207
no of rooms                               0
year of construction                      0
parking space                             0
market                                   28
form of ownership                      2424
dtype: int64